## 8.3 自动并行计算

MXNet后端会自动构建计算图，通过计算图，系统可以知道所有计算的依赖关系，并可以选择将没有依赖关系的多个任务并行执行来获得计算性能的提升。

通常，一个运算符会用到所有CPU或单块GPU上全部的计算资源。例如，dot运算符会用到所有CPU(即使是一台机器上有多个CPU处理器)或单块GPU上所有的线程。如果每个运算符的计算量足够大，只在CPU上或者单块GPU上并行运行多个运算符时，每个运算符的运行只分到CPU或单块GPU上的部分计算资源，即使这些计算可以并行，最终计算性能的提升可能也不明显。本节中探讨的自动并行计算主要关注同时使用CPU和GPU的并行计算，以及计算和通信的并行。

In [2]:
import d2lzh as d2l
import mxnet as mx
from mxnet import nd

### 8.3.1 CPU和GPU的并行计算

In [3]:
def run(x):
    return [nd.dot(x, x) for _ in range(10)]

In [4]:
x_cpu = nd.random_uniform(shape=(2000, 2000))
x_gpu = nd.random_uniform(shape=(6000, 6000), ctx=mx.gpu(0))

MXNetError: [16:57:48] C:\Jenkins\workspace\mxnet-tag\mxnet\src\resource.cc:167: GPU is not enabled

In [5]:
run(x_cpu) # 预热开始
run(x_gpu)
nd.waitall() # 预热结束

with d2l.Benchmark('Run on CPU.'):
    run(x_cpu)
    nd.waitall()
    
with d2l.Benchmark('Then run on GPU.'):
    run(x_gpu)
    nd.waitall()

NameError: name 'x_gpu' is not defined

In [6]:
with d2l.Benchmark('Run on both CPU and GPU in parallel.'):
    run(x_cpu)
    run(x_gpu)
    nd.waitall()

Run on both CPU and GPU in parallel. time: 0.0000 sec


NameError: name 'x_gpu' is not defined

可以看到，当两个计算任务一起执行时，执行总时间小于它们分开执行的总和。这表明，MXNet能有效地在CPU和GPU上自动并行计算。

### 8.3.2 计算和通信的并行计算

在同时使用CPU和GPU的计算中，经常需要在内存和显存之间复制数据，造成数据的通信。

In [7]:
def copy_to_cpu(x):
    return [y.copyto(mx.cpu()) for y in x]

with d2l.Benchmark('Run on GPU.'):
    y = run(x_gpu)
    nd.waitall()
    
with d2l.Benchmark('Then copy to CPU.'):
    copy_to_cpu(y)
    nd.waitall()

Run on GPU. time: 0.0000 sec


NameError: name 'x_gpu' is not defined

In [8]:
with d2l.Benchmark('Run and copy in parallel.'):
    y = run(x_gpu)
    copy_to_cpu(y)
    nd.waitall()

Run and copy in parallel. time: 0.0000 sec


NameError: name 'x_gpu' is not defined

### 小结

MXNet能够通过自动并行计算提升计算性能。例如CPU和GPU的并行计算以及计算和通信的并行。